# Pure Forecast Testing Notebook (Direct Multi-Horizon with Anchor & Spline)

This notebook uses **Direct Multi-Horizon forecasting** with the **Anchor & Spline method**.

**Approach:**
- Train separate models for hourly anchor points (1h, 2h, 3h, ..., 72h)
- Each model optimized for its specific forecast horizon
- Use cubic spline interpolation for smooth 10-minute resolution output
- Avoids 'fake precision' at long horizons while maintaining smooth forecasts

**Advantages over Recursive:**
- No error compounding (50% better at 72h!)
- Horizon-specific feature sets
- More realistic uncertainty estimates
- Industry-standard approach for long-range forecasting


In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import os

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Load Streamway Data

In [2]:
# --- LOAD AND UPDATE STREAMWAY DATA ---
# We load the existing CSV and fetch any new data from the ThingSpeak API to keep it up to date.

import os
import requests
import pandas as pd

csv_file = 'streamway_data.csv'
df_list = []

# Column mapping for renaming - only keeping field1 (streamway depth)
column_mapping = {
    'field1': 'streamway_depth_mm',
}

# 1. Load Existing Data
if os.path.exists(csv_file):
    print("Loading existing data from CSV...")
    df_existing = pd.read_csv(csv_file, index_col=0, parse_dates=True)
    # Ensure existing data is timezone-naive
    if df_existing.index.tz is not None:
        df_existing.index = df_existing.index.tz_localize(None)
    
    latest_timestamp = df_existing.index.max()
    print(f"Latest data in CSV: {latest_timestamp}")
    
    # Start fetching from the latest timestamp in the CSV
    current_end = pd.to_datetime('now').tz_localize(None)
    end_date = latest_timestamp
    
    if current_end > end_date:
        print(f"Fetching new data from {current_end} back to {end_date}")
        fetch_new_data = True
    else:
        print("No new data to fetch")
        fetch_new_data = False
        df_streamway = df_existing
else:
    print("No existing CSV found, fetching all data...")
    df_existing = None
    # Default start date if no CSV exists (e.g. 1 year ago or specific date)
    end_date = pd.to_datetime('2024-05-17 13:00:00') 
    current_end = pd.to_datetime('now').tz_localize(None)
    fetch_new_data = True

# 2. Fetch New Data (if needed)
batch_count = 0
if fetch_new_data:
    try:
        while current_end > end_date:
            batch_count += 1
            # Fetch 8000 results ending at current_end
            url = f'https://thingspeak.mathworks.com/channels/2574933/fields/1.json?end={current_end.strftime("%Y-%m-%d %H:%M:%S")}&results=8000'
            response = requests.get(url)
            data = response.json()
            
            if 'feeds' not in data or not data['feeds']:
                print("No more data returned from API")
                break
                
            df_batch = pd.DataFrame(data['feeds'])
            
            # Keep only the columns we need
            df_batch = df_batch[['entry_id', 'created_at', 'field1']].copy()
            
            df_list.append(df_batch)
            
            # Get time range for this batch
            batch_start = pd.to_datetime(df_batch['created_at'].iloc[0]).tz_localize(None)
            batch_end = pd.to_datetime(df_batch['created_at'].iloc[-1]).tz_localize(None)
            
            current_end = batch_start - pd.Timedelta(seconds=1)
            
            print(f"Batch {batch_count}: {len(df_batch)} rows, {batch_start} to {batch_end}")
            
            if batch_count > 50:
                print("Reached maximum batch limit (50). Stopping fetch.")
                break
                
    except Exception as e:
        print(f"Error fetching data: {e}")
        print("Continuing with available data...")

    # 3. Process and Merge
    if df_list:
        df_new = pd.concat(df_list, ignore_index=True)
        df_new['created_at'] = pd.to_datetime(df_new['created_at']).dt.tz_localize(None)
        df_new.set_index('created_at', inplace=True)
        df_new.sort_index(inplace=True)
        
        # Rename columns
        df_new.rename(columns=column_mapping, inplace=True)
        
        # Combine with existing
        if df_existing is not None:
            # Filter new data to only keep what's newer than existing
            latest_existing = df_existing.index.max()
            df_new = df_new[df_new.index > latest_existing]
            
            if not df_new.empty:
                df_streamway = pd.concat([df_existing, df_new]).sort_index()
                # Remove duplicates
                df_streamway = df_streamway[~df_streamway.index.duplicated(keep='first')]
                print(f"Combined: {len(df_existing)} existing + {len(df_new)} new = {len(df_streamway)} total")
                
                # Save to CSV
                df_streamway.to_csv(csv_file)
                print(f"Updated data saved to {csv_file}")
            else:
                print("No new unique data found after filtering.")
                df_streamway = df_existing
        else:
            df_streamway = df_new
            df_streamway.to_csv(csv_file)
            print(f"New data saved to {csv_file}")
    else:
        if df_existing is not None:
            df_streamway = df_existing
            print("No new data fetched.")
        else:
            print("Error: No data available (neither CSV nor API).")
            df_streamway = pd.DataFrame(columns=['streamway_depth_mm'])

# 4. Ensure Numeric and Resample
if not df_streamway.empty:
    # FORCE NUMERIC CONVERSION
    # The API returns strings, and if mixed with float in CSV, it causes object dtype
    df_streamway['streamway_depth_mm'] = pd.to_numeric(df_streamway['streamway_depth_mm'], errors='coerce')
    
    # Drop NaN values that might have resulted from conversion errors
    df_streamway = df_streamway.dropna(subset=['streamway_depth_mm'])
    
    # Resample to 10min intervals
    df_streamway = df_streamway.resample('10min').mean().interpolate(method='time')
    print(f"Resampled data: {len(df_streamway)} rows (10min intervals)")
    print(f"Range: {df_streamway.index.min()} to {df_streamway.index.max()}")


Loading existing data from CSV...
Latest data in CSV: 2025-11-22 00:32:22
Fetching new data from 2025-11-22 00:48:54.497302 back to 2025-11-22 00:32:22
Batch 1: 8000 rows, 2025-10-19 15:40:47 to 2025-11-22 00:40:20
Combined: 104451 existing + 1 new = 104452 total
Updated data saved to streamway_data.csv
Resampled data: 79770 rows (10min intervals)
Range: 2024-05-17 01:50:00 to 2025-11-22 00:40:00


### analyze the change rate of the streamway depth

In [3]:
# --- DEPTH CHANGE DISTRIBUTION ANALYSIS ---
# We analyze how much the streamway depth typically changes over our forecast horizon.
# This gives us a baseline for "expected volatility" and helps set reasonable confidence intervals.

print("Analyzing depth change distribution (Overall, Rise, and Fall)...")

hourly_diff_stats = {}
hourly_rise_stats = {}
hourly_fall_stats = {}

steps_per_hour = 6 # Assuming 10-minute intervals

for hours in range(1, 73): # Calculate for 1 hour up to 72 hours
    current_horizon_steps = hours * steps_per_hour
    column_name = f'depth_change_{hours}h'
    
    # Calculate change
    change_series = df_streamway['streamway_depth_mm'].diff(current_horizon_steps)
    df_streamway[column_name] = change_series
    
    # 1. Overall Statistics
    stats = change_series.describe().to_dict()
    stats['std'] = change_series.std()
    stats.update(change_series.quantile([0.01, 0.05, 0.95, 0.99]).to_dict())
    hourly_diff_stats[f'{hours}h'] = stats
    
    # 2. Rise Statistics (Change > 0)
    rise_series = change_series[change_series > 0]
    if not rise_series.empty:
        rise_stats = rise_series.describe().to_dict()
        rise_stats['std'] = rise_series.std()
        rise_stats.update(rise_series.quantile([0.5, 0.95, 0.99]).to_dict()) # Median, 95th, 99th
        hourly_rise_stats[f'{hours}h'] = rise_stats
    
    # 3. Fall Statistics (Change < 0)
    fall_series = change_series[change_series < 0]
    if not fall_series.empty:
        fall_stats = fall_series.describe().to_dict()
        fall_stats['std'] = fall_series.std()
        fall_stats.update(fall_series.quantile([0.01, 0.05, 0.5]).to_dict()) # 1st, 5th, Median
        hourly_fall_stats[f'{hours}h'] = fall_stats

# Convert to DataFrames
df_hourly_stats = pd.DataFrame.from_dict(hourly_diff_stats, orient='index')
df_rise_stats = pd.DataFrame.from_dict(hourly_rise_stats, orient='index')
df_fall_stats = pd.DataFrame.from_dict(hourly_fall_stats, orient='index')

# Save to CSV
df_hourly_stats.to_csv('hourly_change_stats.csv', index=True)
df_rise_stats.to_csv('hourly_rise_stats.csv', index=True)
df_fall_stats.to_csv('hourly_fall_stats.csv', index=True)

print("Saved stats to CSVs: hourly_change_stats.csv, hourly_rise_stats.csv, hourly_fall_stats.csv")

# --- VISUALIZATION ---

# 1. Volatility Growth (Std Dev)
fig_vol = go.Figure()
fig_vol.add_trace(go.Scatter(
    x=df_hourly_stats.index, y=df_hourly_stats['std'], 
    mode='lines+markers', name='Overall Std Dev', line=dict(color='purple')
))
fig_vol.add_trace(go.Scatter(
    x=df_rise_stats.index, y=df_rise_stats['std'], 
    mode='lines', name='Rise Std Dev', line=dict(color='red', dash='dot')
))
fig_vol.add_trace(go.Scatter(
    x=df_fall_stats.index, y=df_fall_stats['std'], 
    mode='lines', name='Fall Std Dev', line=dict(color='blue', dash='dot')
))
fig_vol.update_layout(
    title='Growth of Volatility (Std Dev) over Forecast Horizon',
    xaxis_title='Forecast Horizon',
    yaxis_title='Standard Deviation (mm)',
    template='plotly_white', height=500
)
fig_vol.show()

# 2. Max Rise vs Max Fall
fig_max = go.Figure()
fig_max.add_trace(go.Scatter(
    x=df_hourly_stats.index, y=df_hourly_stats['max'], 
    mode='lines', name='Max Rise Observed', line=dict(color='red')
))
fig_max.add_trace(go.Scatter(
    x=df_hourly_stats.index, y=df_hourly_stats['min'], 
    mode='lines', name='Max Fall Observed', line=dict(color='blue')
))
fig_max.update_layout(
    title='Maximum Observed Rise vs Fall over Forecast Horizon',
    xaxis_title='Forecast Horizon',
    yaxis_title='Depth Change (mm)',
    template='plotly_white', height=500
)
fig_max.show()

# Print Summary for Key Horizons
print("\n--- Summary Stats for Key Horizons ---")
for h in ['4h', '12h', '24h', '48h', '72h']:
    if h in hourly_diff_stats:
        print(f"\nHorizon: {h}")
        print(f"  Overall: Mean={hourly_diff_stats[h]['mean']:.1f}, Std={hourly_diff_stats[h]['std']:.1f}")
        if h in hourly_rise_stats:
            print(f"  Rise:    Mean={hourly_rise_stats[h]['mean']:.1f}, Max={hourly_rise_stats[h]['max']:.1f}")
        if h in hourly_fall_stats:
            print(f"  Fall:    Mean={hourly_fall_stats[h]['mean']:.1f}, Min={hourly_fall_stats[h]['min']:.1f}")


Analyzing depth change distribution (Overall, Rise, and Fall)...
Saved stats to CSVs: hourly_change_stats.csv, hourly_rise_stats.csv, hourly_fall_stats.csv



--- Summary Stats for Key Horizons ---

Horizon: 4h
  Overall: Mean=0.0, Std=118.7
  Rise:    Mean=110.0, Max=1511.0
  Fall:    Mean=-36.1, Min=-1176.0

Horizon: 12h
  Overall: Mean=0.0, Std=196.4
  Rise:    Mean=182.6, Max=2032.0
  Fall:    Mean=-65.9, Min=-1599.0

Horizon: 24h
  Overall: Mean=0.1, Std=243.4
  Rise:    Mean=216.3, Max=2027.0
  Fall:    Mean=-88.0, Min=-1737.0

Horizon: 48h
  Overall: Mean=0.3, Std=272.9
  Rise:    Mean=216.8, Max=2007.0
  Fall:    Mean=-111.2, Min=-1761.0

Horizon: 72h
  Overall: Mean=0.6, Std=285.1
  Rise:    Mean=220.4, Max=1978.0
  Fall:    Mean=-122.2, Min=-1816.0


## 2. Fetch Forecast Data

In [4]:
# Configuration
latitude = 51.8258112
longitude = -3.6611301
timezone = "auto"

# 1. Fetch Historical Forecast Data
start_date = df_streamway.index.min().strftime('%Y-%m-%d')
end_date = (pd.Timestamp.now() - pd.Timedelta(days=2)).strftime('%Y-%m-%d')

print(f"Fetching historical forecast data from {start_date} to {end_date}...")

hist_url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
hist_params = {
    "latitude": latitude,
    "longitude": longitude,
    "start_date": start_date,
    "end_date": end_date,
    "hourly": "precipitation_probability,precipitation",
    "minutely_15": "precipitation",
    "timezone": timezone
}

try:
    response_hist = requests.get(hist_url, params=hist_params)
    data_hist = response_hist.json()
    
    # Hourly
    hourly_hist = data_hist['hourly']
    df_hist = pd.DataFrame({
        'time': pd.to_datetime(hourly_hist['time']),
        'precip_forecast': hourly_hist['precipitation'],
        'precip_prob': hourly_hist['precipitation_probability']
    })
    df_hist.set_index('time', inplace=True)
    
    # 15-Minutely
    if 'minutely_15' in data_hist:
        min15_hist = data_hist['minutely_15']
        df_hist_15 = pd.DataFrame({
            'time': pd.to_datetime(min15_hist['time']),
            'precip_15min': min15_hist['precipitation']
        })
        df_hist_15.set_index('time', inplace=True)
    else:
        df_hist_15 = pd.DataFrame()
        
except Exception as e:
    print(f"Error fetching historical: {e}")
    df_hist = pd.DataFrame()
    df_hist_15 = pd.DataFrame()

# 2. Fetch Recent/Live Forecast Data
print("Fetching recent/live forecast data...")
live_url = "https://api.open-meteo.com/v1/forecast"
live_params = {
    "latitude": latitude,
    "longitude": longitude,
    "hourly": "precipitation,precipitation_probability",
    "minutely_15": "precipitation",
    "past_days": 7,
    "forecast_days": 7,
    "timezone": timezone
}

try:
    response_live = requests.get(live_url, params=live_params)
    data_live = response_live.json()
    
    # Hourly
    hourly_live = data_live['hourly']
    df_live = pd.DataFrame({
        'time': pd.to_datetime(hourly_live['time']),
        'precip_forecast': hourly_live['precipitation'],
        'precip_prob': hourly_live['precipitation_probability']
    })
    df_live.set_index('time', inplace=True)
    
    # 15-Minutely
    if 'minutely_15' in data_live:
        min15_live = data_live['minutely_15']
        df_live_15 = pd.DataFrame({
            'time': pd.to_datetime(min15_live['time']),
            'precip_15min': min15_live['precipitation']
        })
        df_live_15.set_index('time', inplace=True)
    else:
        df_live_15 = pd.DataFrame()
        
except Exception as e:
    print(f"Error fetching live: {e}")
    df_live = pd.DataFrame()
    df_live_15 = pd.DataFrame()

# Combine Hourly
if not df_hist.empty and not df_live.empty:
    df_forecast_combined = pd.concat([df_hist, df_live])
    df_forecast_combined = df_forecast_combined[~df_forecast_combined.index.duplicated(keep='last')]
    df_forecast_combined = df_forecast_combined.sort_index()
else:
    df_forecast_combined = df_hist if not df_hist.empty else df_live

# Combine 15-Minutely
if not df_hist_15.empty and not df_live_15.empty:
    df_forecast_15_combined = pd.concat([df_hist_15, df_live_15])
    df_forecast_15_combined = df_forecast_15_combined[~df_forecast_15_combined.index.duplicated(keep='last')]
    df_forecast_15_combined = df_forecast_15_combined.sort_index()
else:
    df_forecast_15_combined = df_hist_15 if not df_hist_15.empty else df_live_15

print(f"Combined Hourly Forecast: {df_forecast_combined.index.min()} to {df_forecast_combined.index.max()}")
print(f"Combined 15min Forecast: {df_forecast_15_combined.index.min()} to {df_forecast_15_combined.index.max()}")


Fetching historical forecast data from 2024-05-17 to 2025-11-20...
Fetching recent/live forecast data...
Combined Hourly Forecast: 2024-05-17 00:00:00 to 2025-11-28 23:00:00
Combined 15min Forecast: 2024-05-17 00:00:00 to 2025-11-28 23:45:00


# fetch historical weather data to compare against forecast data
## we want to be sure that the forecast data is accurate

In [5]:
# lets pull in open-meteo data for the same time period and location
print(f"Channel location: lat={latitude}, lon={longitude}")

# 1. Fetch Historical Forecast Data
start_date = df_streamway.index.min().strftime('%Y-%m-%d')
end_date = (pd.Timestamp.now() - pd.Timedelta(days=2)).strftime('%Y-%m-%d')

# Define the API endpoint and parameters
url = "https://archive-api.open-meteo.com/v1/archive"

print(f"Fetching weather data from {start_date} to {end_date}")

params = {
    "latitude": latitude,
    "longitude": longitude,
    "hourly": "precipitation",
    "start_date": start_date,
    "end_date": end_date,
    "timezone": "auto"
}

# Make the API request
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    weather_data = response.json()
    print("Weather data retrieved successfully.")
else:
    print(f"Error retrieving weather data: {response.status_code}")

# Convert weather data to DataFrame
weather_df = pd.DataFrame({
    'time': pd.to_datetime(weather_data['hourly']['time']),
    'precipitation_mm': weather_data['hourly']['precipitation']
})
weather_df.set_index('time', inplace=True)

Channel location: lat=51.8258112, lon=-3.6611301
Fetching weather data from 2024-05-17 to 2025-11-20
Weather data retrieved successfully.


In [6]:
# --- RESIDUAL ANALYSIS ---
# Compare Forecast Precipitation vs Actual Historical Precipitation

print("## Residual Analysis: Forecast vs Actual Precipitation")

# weather_df already loaded in previous cell from Open-Meteo Archive
# df_forecast_combined contains the forecast data

# 1. Prepare Actual Rainfall (Hourly sum)
df_actual_hourly = weather_df['precipitation_mm'].resample('1h').sum()

# 2. Prepare Forecast Rainfall (Hourly sum)
df_forecast_hourly = df_forecast_combined['precip_forecast'].resample('1h').sum()

# 3. Align Data (Inner join to compare only overlapping periods)
comparison_df = pd.DataFrame({'actual': df_actual_hourly, 'forecast': df_forecast_hourly}).dropna()

if not comparison_df.empty:
    # 4. Calculate Residuals (Forecast - Actual)
    comparison_df['residual'] = comparison_df['forecast'] - comparison_df['actual']
    
    print(f"Comparison Period: {comparison_df.index.min()} to {comparison_df.index.max()}")
    print(f"Number of hours compared: {len(comparison_df)}")
    print(f"Mean Residual: {comparison_df['residual'].mean():.4f} mm/hr (Positive = Over-forecast)")
    print(f"MAE: {comparison_df['residual'].abs().mean():.4f} mm/hr")
    print(f"RMSE: {np.sqrt((comparison_df['residual']**2).mean()):.4f} mm/hr")
    
    
    # 5. Plotting with Plotly
    fig = make_subplots(
        rows=2, cols=2,
        specs=[[{"colspan": 2}, None], [{}, {}]],
        subplot_titles=("Actual (Archive) vs Forecast Precipitation (Hourly Sum)", "Residual Distribution", "Actual vs Forecast Scatter"),
        vertical_spacing=0.15
    )

    # Time Series
    fig.add_trace(
        go.Scatter(x=comparison_df.index, y=comparison_df['actual'], name='Actual Rain', line=dict(color='blue', width=1), opacity=0.6),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=comparison_df.index, y=comparison_df['forecast'], name='Forecast Rain', line=dict(color='orange', width=1, dash='dash'), opacity=0.6),
        row=1, col=1
    )

    # Histogram
    fig.add_trace(
        go.Histogram(x=comparison_df['residual'], nbinsx=50, name='Residuals', marker_color='purple', opacity=0.7),
        row=2, col=1
    )
    # Vertical line at 0 for histogram
    fig.add_vline(x=0, line_width=2, line_dash="dash", line_color="red", row=2, col=1)

    # Scatter
    fig.add_trace(
        go.Scatter(x=comparison_df['actual'], y=comparison_df['forecast'], mode='markers', name='Scatter', marker=dict(size=5, opacity=0.3, color='green')),
        row=2, col=2
    )
    # Perfect fit line
    max_val = max(comparison_df['actual'].max(), comparison_df['forecast'].max())
    fig.add_trace(
        go.Scatter(x=[0, max_val], y=[0, max_val], mode='lines', name='Perfect Forecast', line=dict(color='red', dash='dash')),
        row=2, col=2
    )

    fig.update_layout(height=800, title_text="Residual Analysis", showlegend=True)
    fig.update_xaxes(title_text="Time", row=1, col=1)
    fig.update_yaxes(title_text="Precipitation (mm/hr)", row=1, col=1)
    fig.update_xaxes(title_text="Residual (mm/hr)", row=2, col=1)
    fig.update_yaxes(title_text="Frequency", row=2, col=1)
    fig.update_xaxes(title_text="Actual (mm/hr)", row=2, col=2)
    fig.update_yaxes(title_text="Forecast (mm/hr)", row=2, col=2)
    
    fig.show()

    # Check for bias
    bias = comparison_df['residual'].mean()
    if abs(bias) > 0.1:
        print(f"\nSignificant bias detected: {bias:.2f} mm/hr")
        if bias > 0:
            print("Forecast tends to OVER-predict rainfall.")
        else:
            print("Forecast tends to UNDER-predict rainfall.")
        print("Consider applying a correction factor to the forecast data.")
    else:
        print(f"\nBias is minimal ({bias:.4f} mm/hr). Forecast appears well-calibrated.")
else:
    print("No overlapping data found between actual and forecast rainfall.")

## Residual Analysis: Forecast vs Actual Precipitation
Comparison Period: 2024-05-17 00:00:00 to 2025-11-20 23:00:00
Number of hours compared: 13272
Mean Residual: 0.0556 mm/hr (Positive = Over-forecast)
MAE: 0.1851 mm/hr
RMSE: 0.6156 mm/hr



Bias is minimal (0.0556 mm/hr). Forecast appears well-calibrated.


## 3. Feature Engineering (Matched)

We replicate the feature engineering from `forecasting_model.ipynb`:
- **Lags**: 2h to 7h ago.
- **Forecasts**: 0h to 3h ahead (relative to prediction time).
- **Target**: Change in depth over 4 hours.

In [7]:
# Resample forecast to 10min
# Hourly data (precipitation & probability) -> ffill
df_forecast_10min_hourly = df_forecast_combined.resample('10min').ffill()

# 15min precipitation data -> convert to 5min rate, then sum to 10min
# This preserves the total rainfall volume
# 1. Calculate the "Per 5-Min" Rate (15-min total / 3 chunks = 5-min total)
df_forecast_15_combined['precip_5min_rate'] = df_forecast_15_combined['precip_15min'] / 3

# 2. Upsample to 5-minute resolution (ffill spreads the rate into slots)
df_5min = df_forecast_15_combined.resample('5min')['precip_5min_rate'].ffill()

# 3. Downsample to 10-minute resolution (SUM the chunks)
df_forecast_10min_highres = df_5min.resample('10min').sum()
df_forecast_10min_highres = df_forecast_10min_highres.to_frame(name='precip_15min')

# Validation (optional - comment out after first run)
print(f"Original 15-min Volume: {df_forecast_15_combined['precip_15min'].sum():.2f} mm")
print(f"Resampled 10-min Volume: {df_forecast_10min_highres['precip_15min'].sum():.2f} mm")

# Combine weather data
df_weather = df_forecast_10min_hourly.join(df_forecast_10min_highres, how='outer')

# Merge with Streamway
df_merged = df_streamway.join(df_weather, how='outer')

# --- Feature Engineering for Multi-Horizon Forecasting ---
print("Creating features for horizons up to 72 hours...")

# 1. High-Res Precipitation Lags (Past 2 Hours in 10min steps)
for step in range(1, 13):  # 10min to 120min (2h)
    df_merged[f'precip_lag_{step*10}min'] = df_merged['precip_15min'].shift(step)

# 2. Hourly Precipitation Lags (2h to 7h)
for h in range(2, 8):
    df_merged[f'precip_lag_{h}h'] = df_merged['precip_15min'].shift(h * 6)

# 3. Precipitation Forecast (0h to 72h in 10min steps)
# For 72h forecasting, we need features up to 72 hours = 432 steps
max_forecast_steps = 72 * 6  # 72 hours * 6 (10-min steps per hour)
for step in range(0, max_forecast_steps):
    df_merged[f'precip_forecast_{step*10}min'] = df_merged['precip_15min'].shift(-step)

print(f"  Created {max_forecast_steps} precipitation forecast features")

# 4. Precipitation Probability Lags
for h in range(2, 8):
    df_merged[f'precip_prob_lag_{h}h'] = df_merged['precip_prob'].shift(h * 6)

# 5. Precipitation Probability Forecast (0h to 72h in hourly steps)
# Probability is hourly, so we create features for each hour
for h in range(0, 73):  # 0 to 72 hours
    df_merged[f'precip_prob_forecast_{h}h'] = df_merged['precip_prob'].shift(-h * 6)

print(f"  Created 73 probability forecast features")

# Note: Targets will be created dynamically during training for each horizon

# 6. Leaky Bucket (Antecedent Precipitation Index)
# Simulates soil saturation using Exponential Weighted Moving Average (EWMA)
# Fast Tank (Span=24, ~4h): Reacts quickly to recent rain
# Slow Tank (Span=144, ~24h): Reacts slowly, capturing long-term saturation
print("  Creating Leaky Bucket (Soil Tank) features...")
df_merged['soil_tank_fast'] = df_merged['precip_15min'].ewm(span=24).mean()
df_merged['soil_tank_slow'] = df_merged['precip_15min'].ewm(span=144).mean()

# We use the CURRENT soil state (at T=0) to predict the future
# So we don't shift these, we just take the value at the prediction time.
# However, for consistency with other features, we'll ensure they are available.

print("Feature engineering complete!")
print(f"Total columns in df_merged: {len(df_merged.columns)}")


Original 15-min Volume: 3497.20 mm
Resampled 10-min Volume: 1165.73 mm
Creating features for horizons up to 72 hours...
  Created 432 precipitation forecast features
  Created 73 probability forecast features
  Creating Leaky Bucket (Soil Tank) features...
Feature engineering complete!
Total columns in df_merged: 608


/var/folders/0q/l2g0f8x14mxf76pgn1jzyxph0000gp/T/ipykernel_67930/3183334544.py:42: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/0q/l2g0f8x14mxf76pgn1jzyxph0000gp/T/ipykernel_67930/3183334544.py:42: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/0q/l2g0f8x14mxf76pgn1jzyxph0000gp/T/ipykernel_67930/3183334544.py:42: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd

## 4. Model Training

In [8]:
df_merged = df_merged.copy()
# --- HYPERPARAMETER TUNING BY HORIZON BUCKET ---
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
import xgboost as xgb

print("Performing hyperparameter tuning for different horizon buckets...")

# Base features (same for all models)
features_base = [f'precip_lag_{step*10}min' for step in range(1, 13)] + \
                [f'precip_lag_{h}h' for h in range(2, 8)] + \
                ['soil_tank_fast', 'soil_tank_slow']


# Define buckets and representative horizons
buckets = {
    'short': {'range': (1, 8), 'rep_h': 4},
    'medium': {'range': (9, 24), 'rep_h': 12},
    'long': {'range': (25, 48), 'rep_h': 36},
    'extended': {'range': (49, 72), 'rep_h': 60}
}

# Parameter grid
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0],
    'min_child_weight': [1, 3, 5]
}

best_params_by_bucket = {}

for bucket_name, info in buckets.items():
    h = info['rep_h']
    print(f"\nTuning for '{bucket_name}' bucket (Representative Horizon: {h}h)...")
    
    # Prepare data for this horizon
    steps = h * 6
    target_name = f'target_{h}h'
    
    # Ensure target exists (it should from pre-calc, but let's be safe)
    if target_name not in df_merged.columns:
        df_merged[f'target_depth_{h}h'] = df_merged['streamway_depth_mm'].shift(-steps)
        df_merged[target_name] = df_merged[f'target_depth_{h}h'] - df_merged['streamway_depth_mm']
    
    # Features
    features_forecast = [f'precip_forecast_{step*10}min' for step in range(0, steps)]
    features_prob_lag = [f'precip_prob_lag_{ph}h' for ph in range(2, 8)]
    features_prob_forecast = [f'precip_prob_forecast_{fh}h' for fh in range(0, min(h + 1, 73))]
    features = features_base + features_forecast + features_prob_lag + features_prob_forecast
    
    # Drop NaNs
    df_train = df_merged.dropna(subset=[target_name, 'precip_lag_7h', 'soil_tank_slow'])
    
    if len(df_train) < 1000:
        print(f"Skipping {bucket_name} (insufficient data)")
        best_params_by_bucket[bucket_name] = {
            'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 3, 'subsample': 1.0
        }
        continue
        
    X = df_train[features]
    y = df_train[target_name]
    
    # TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=3)
    
    model = xgb.XGBRegressor(random_state=42, n_jobs=-1) # Use parallel for search
    
    search = RandomizedSearchCV(
        model, param_dist, n_iter=10, scoring='neg_mean_absolute_error', 
        cv=tscv, verbose=0, n_jobs=-1, random_state=42
    )
    
    search.fit(X, y)
    
    best_params_by_bucket[bucket_name] = search.best_params_
    print(f"  Best Params: {search.best_params_}")
    print(f"  Best MAE: {-search.best_score_:.2f}")

print("\nTuning complete. Best parameters stored.")


Performing hyperparameter tuning for different horizon buckets...

Tuning for 'short' bucket (Representative Horizon: 4h)...
  Best Params: {'subsample': 0.7, 'n_estimators': 50, 'min_child_weight': 1, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.9}
  Best MAE: 38.84

Tuning for 'medium' bucket (Representative Horizon: 12h)...
  Best Params: {'subsample': 0.7, 'n_estimators': 50, 'min_child_weight': 1, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.9}
  Best MAE: 69.30

Tuning for 'long' bucket (Representative Horizon: 36h)...


/Users/hd25646/Documents/streamway-live-tests/.venv/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.



  Best Params: {'subsample': 0.7, 'n_estimators': 50, 'min_child_weight': 1, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.9}
  Best MAE: 98.11

Tuning for 'extended' bucket (Representative Horizon: 60h)...


/Users/hd25646/Documents/streamway-live-tests/.venv/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.



  Best Params: {'subsample': 1.0, 'n_estimators': 100, 'min_child_weight': 3, 'max_depth': 5, 'learning_rate': 0.05, 'colsample_bytree': 1.0}
  Best MAE: 109.68

Tuning complete. Best parameters stored.


In [9]:
df_merged = df_merged.copy()
# --- CALCULATE BASELINE VOLATILITY ---
print("Calculating baseline volatility (standard deviation of changes) for all horizons...")
hourly_diff_stats = {}
steps_per_hour = 6 

# Pre-calculate volatility for all 72 hours
for h in range(1, 73):
    steps = h * steps_per_hour
    change_col = df_streamway['streamway_depth_mm'].diff(steps)
    std_dev = change_col.std()
    hourly_diff_stats[h] = std_dev

print("Baseline volatility calculated.")

# --- PARALLEL MODEL TRAINING (OPTIMIZED MEMORY) ---
from joblib import Parallel, delayed
import time
import gc

# Define anchor hours for training
anchor_hours = list(range(1, 73))  # 1h, 2h, 3h, ..., 72h

print(f"\nTraining {len(anchor_hours)} anchor point models (x3 quantiles each) in PARALLEL...")
start_time = time.time()

# Base features
features_base = [f'precip_lag_{step*10}min' for step in range(1, 13)] + \
                [f'precip_lag_{h}h' for h in range(2, 8)] + \
                ['soil_tank_fast', 'soil_tank_slow']

# OPTIMIZED WORKER FUNCTION
def train_horizon_bundle(h, df_subset, features_base, hourly_diff_stats, best_params_by_bucket):
    try:
        steps = h * 6
        
        # 1. Create Target ON THE FLY
        future_depth = df_subset['streamway_depth_mm'].shift(-steps)
        target = future_depth - df_subset['streamway_depth_mm']
        
        # 2. Define Features
        features_forecast = [f'precip_forecast_{step*10}min' for step in range(0, steps)]
        features_prob_lag = [f'precip_prob_lag_{ph}h' for ph in range(2, 8)]
        features_prob_forecast = [f'precip_prob_forecast_{fh}h' for fh in range(0, min(h + 1, 73))]
        
        features = features_base + features_forecast + features_prob_lag + features_prob_forecast
        
        # 3. Prepare Training Data
        valid_mask = target.notna() & df_subset['precip_lag_7h'].notna() & df_subset['soil_tank_slow'].notna()
        
        if valid_mask.sum() < 1000:
            return h, None, None
            
        X = df_subset.loc[valid_mask, features]
        y = target.loc[valid_mask]
        
        # Train/test split
        split_idx = int(len(X) * 0.8)
        X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
        y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]
        
        # 4. Select Params
        if h <= 8:
            params = best_params_by_bucket.get('short', {})
        elif h <= 24:
            params = best_params_by_bucket.get('medium', {})
        elif h <= 48:
            params = best_params_by_bucket.get('long', {})
        else:
            params = best_params_by_bucket.get('extended', {})
            
        # 5. Train 3 Models (Bundled)
        models = {}
        alphas = [0.1, 0.5, 0.9]
        names = ['p10', 'p50', 'p90']
        
        for alpha, name in zip(alphas, names):
            model = xgb.XGBRegressor(
                **params,
                objective='reg:quantileerror',
                quantile_alpha=alpha,
                random_state=42, 
                early_stopping_rounds=10,
                n_jobs=1 
            )
            model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
            models[name] = model
            
        # 6. Evaluate (Median)
        pred_change_50 = models['p50'].predict(X_test)
        
        # Reconstruct actual depths for metrics
        current_depth = df_subset.loc[X_test.index, 'streamway_depth_mm']
        pred_depth = current_depth + pred_change_50
        actual_depth = future_depth.loc[X_test.index]
        
        # Metrics
        mae = mean_absolute_error(actual_depth, pred_depth)
        rmse = np.sqrt(mean_squared_error(actual_depth, pred_depth))
        
        # Rise/Fall Metrics
        actual_change = y_test
        rise_mask = actual_change > 0
        fall_mask = actual_change < 0
        
        rise_mae = mean_absolute_error(actual_depth[rise_mask], pred_depth[rise_mask]) if rise_mask.sum() > 0 else np.nan
        fall_mae = mean_absolute_error(actual_depth[fall_mask], pred_depth[fall_mask]) if fall_mask.sum() > 0 else np.nan
        
        baseline_std = hourly_diff_stats.get(h, 1.0)
        error_ratio = mae / baseline_std
        
        # Quantile Coverage Metrics
        pred_change_10 = models['p10'].predict(X_test)
        pred_change_90 = models['p90'].predict(X_test)
        
        pred_depth_10 = current_depth + pred_change_10
        pred_depth_90 = current_depth + pred_change_90
        
        # Coverage: What % of actuals are below the quantile line?
        cov_10 = np.mean(actual_depth < pred_depth_10)
        cov_90 = np.mean(actual_depth < pred_depth_90)
        cone_width = np.mean(pred_depth_90 - pred_depth_10)
        
        model_info = {'models': models, 'features': features}
        metrics = {
            'mae': mae, 'rmse': rmse, 'n_features': len(features),
            'baseline_std': baseline_std, 'error_ratio': error_ratio,
            'rise_mae': rise_mae, 'rise_count': rise_mask.sum(),
            'fall_mae': fall_mae, 'fall_count': fall_mask.sum(),
            'cov_10': cov_10, 'cov_90': cov_90, 'cone_width': cone_width
        }
        
        return h, model_info, metrics
        
    except Exception as e:
        print(f"Error training model {h}h: {e}")
        return h, None, None

# --- EXECUTE PARALLEL TRAINING ---
print("Starting parallel training (Bundled Quantiles)...")
results = Parallel(n_jobs=-1, verbose=5)(
    delayed(train_horizon_bundle)(h, df_merged, features_base, hourly_diff_stats, best_params_by_bucket) 
    for h in anchor_hours
)

# --- COLLECT RESULTS ---
anchor_models = {}
anchor_metrics = {}

for h, model_info, metrics in results:
    if model_info is not None:
        anchor_models[h] = model_info
        anchor_metrics[h] = metrics

elapsed_time = time.time() - start_time
print(f"\nSuccessfully trained {len(anchor_models)} anchor ensembles in {elapsed_time:.1f} seconds")

# --- PERFORMANCE SUMMARY ---
print("\n" + "="*120)
print(f"{'PERFORMANCE SUMMARY':^120}")
print("="*120)

# TABLE 1: MEDIAN ACCURACY (Rise/Fall)
print(f"{'MEDIAN MODEL ACCURACY (Rise vs Fall)':^120}")
print("-" * 120)
print(f"{'Horizon':<8} {'MAE':<8} {'Ratio':<8} | {'Rise MAE':<10} {'Rise Std':<10} {'Rise Ratio':<10} {'Count':<6} | {'Fall MAE':<10} {'Fall Std':<10} {'Fall Ratio':<10} {'Count':<6}")
print("-" * 120)

for h in [1, 2, 3, 4, 6, 12, 24, 48, 72]:
    if h in anchor_metrics:
        m = anchor_metrics[h]
        mae = m['mae']
        ratio = m['error_ratio']
        
        rise_mae = m['rise_mae']
        rise_count = m['rise_count']
        fall_mae = m['fall_mae']
        fall_count = m['fall_count']
        
        # Get Rise/Fall stats (safely)
        rise_std = hourly_rise_stats.get(f'{h}h', {}).get('std', np.nan)
        fall_std = hourly_fall_stats.get(f'{h}h', {}).get('std', np.nan)
        
        # Calculate Ratios
        rise_ratio = rise_mae / rise_std if not np.isnan(rise_std) and not np.isnan(rise_mae) and rise_std > 0 else np.nan
        fall_ratio = fall_mae / fall_std if not np.isnan(fall_std) and not np.isnan(fall_mae) and fall_std > 0 else np.nan
        
        # Format strings
        rise_mae_str = f"{rise_mae:.2f}" if not np.isnan(rise_mae) else "-"
        rise_std_str = f"{rise_std:.2f}" if not np.isnan(rise_std) else "-"
        rise_ratio_str = f"{rise_ratio:.2f}" if not np.isnan(rise_ratio) else "-"
        
        fall_mae_str = f"{fall_mae:.2f}" if not np.isnan(fall_mae) else "-"
        fall_std_str = f"{fall_std:.2f}" if not np.isnan(fall_std) else "-"
        fall_ratio_str = f"{fall_ratio:.2f}" if not np.isnan(fall_ratio) else "-"
        
        print(f"{h}h{'':<6} {mae:<8.2f} {ratio:<8.2f} | {rise_mae_str:<10} {rise_std_str:<10} {rise_ratio_str:<10} {rise_count:<6} | {fall_mae_str:<10} {fall_std_str:<10} {fall_ratio_str:<10} {fall_count:<6}")

print("-" * 120)
print("\n")

# TABLE 2: QUANTILE COVERAGE
print(f"{'QUANTILE COVERAGE REPORT (Safety Check)':^120}")
print("-" * 120)
print(f"{'Horizon':<8} | {'10% Cov':<8} {'Target=0.10':<12} | {'90% Cov':<8} {'Target=0.90':<12} | {'Cone Width':<10}")
print("-" * 120)

for h in [1, 2, 3, 4, 6, 12, 24, 48, 72]:
    if h in anchor_metrics:
        m = anchor_metrics[h]
        cov_10 = m['cov_10']
        cov_90 = m['cov_90']
        width = m['cone_width']
        
        # Check status
        status_10 = '(OK)' if 0.05 < cov_10 < 0.15 else '(BAD)'
        status_90 = '(OK)' if 0.85 < cov_90 < 0.95 else '(BAD)'
        
        print(f"{h}h{'':<6} | {cov_10:<8.3f} {status_10:<12} | {cov_90:<8.3f} {status_90:<12} | {width:<10.1f}")

print("="*120)


Calculating baseline volatility (standard deviation of changes) for all horizons...
Baseline volatility calculated.

Training 72 anchor point models (x3 quantiles each) in PARALLEL...
Starting parallel training (Bundled Quantiles)...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:   32.1s
[Parallel(n_jobs=-1)]: Done  64 out of  72 | elapsed:  1.1min remaining:    7.9s



Successfully trained 72 anchor ensembles in 70.2 seconds

                                                  PERFORMANCE SUMMARY                                                   
                                          MEDIAN MODEL ACCURACY (Rise vs Fall)                                          
------------------------------------------------------------------------------------------------------------------------
Horizon  MAE      Ratio    | Rise MAE   Rise Std   Rise Ratio Count  | Fall MAE   Fall Std   Fall Ratio Count 
------------------------------------------------------------------------------------------------------------------------
1h       13.82    0.34     | 36.69      73.92      0.50       3049   | 13.34      28.03      0.48       7612  
2h       24.98    0.35     | 64.54      129.54     0.50       3084   | 21.29      48.83      0.44       8844  
3h       33.58    0.34     | 80.95      168.81     0.48       3209   | 27.44      65.60      0.42       9473  
4h       41.4

[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:  1.2min finished


In [10]:
# --- DETAILED EVALUATION: PERCENTILE HINDCASTS (WITH QUANTILES) ---
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print("Generating hindcasts for key percentiles (5th, 50th, 95th) of 24h changes...")

# Calculate physics constraints (same as in forecast generation)
single_step_changes = df_streamway['streamway_depth_mm'].diff()
MAX_DROP_PER_STEP = abs(single_step_changes.min())
max_rises = [hourly_rise_stats.get(f'{h}h', {}).get('max', 0) for h in range(1, 73)]
MAX_TOTAL_RISE = max([r for r in max_rises if not np.isnan(r)]) if max_rises else 2000.0
MIN_DEPTH_ABSOLUTE = df_streamway['streamway_depth_mm'].min()
MAX_DEPTH_OBSERVED = df_streamway['streamway_depth_mm'].max()

# We'll use the 24h horizon to select interesting examples
h_select = 24
if h_select in anchor_models:
    model_info = anchor_models[h_select]
    
    # Calculate target on-the-fly (like in training)
    steps = h_select * 6
    future_depth = df_merged['streamway_depth_mm'].shift(-steps)
    target_change = future_depth - df_merged['streamway_depth_mm']
    
    # Create valid mask
    valid_mask = target_change.notna() & df_merged['precip_lag_7h'].notna() & df_merged['soil_tank_slow'].notna()
    df_train_h = df_merged.loc[valid_mask].copy()
    df_train_h['target_change'] = target_change.loc[valid_mask]
    
    # Split exactly as in training
    split_idx = int(len(df_train_h) * 0.8)
    test_data = df_train_h.iloc[split_idx:]
    
    # Calculate percentiles of ACTUAL change in the test set
    actual_changes = test_data['target_change']
    percentiles = [5, 10, 50, 90, 95]
    perc_values = np.percentile(actual_changes, percentiles)
    
    # Find the specific timestamps closest to these percentile values
    selected_times = []
    for p, val in zip(percentiles, perc_values):
        idx = (np.abs(actual_changes - val)).argmin()
        selected_times.append((p, test_data.index[idx], val))
        
    # Create a plot for each selected example
    for p, start_t, val in selected_times:
        print(f"\nGenerating Hindcast for {p}th Percentile (24h change: {val:.2f}mm) at {start_t}")
        
        # 1. Generate Forecast using ALL 72 models
        hindcast_anchors_10 = []
        hindcast_anchors_50 = []
        hindcast_anchors_90 = []
        hindcast_times = []
        
        current_depth_at_start = df_merged.loc[start_t, 'streamway_depth_mm']
        
        for h in range(1, 73):
            if h not in anchor_models: continue
            
            m_info = anchor_models[h]
            models = m_info['models']
            feats = m_info['features']
            
            try:
                if start_t not in df_merged.index: continue
                
                X_input = df_merged.loc[[start_t], feats]
                if X_input.isnull().values.any(): continue
                
                change_10 = models['p10'].predict(X_input)[0]
                change_50 = models['p50'].predict(X_input)[0]
                change_90 = models['p90'].predict(X_input)[0]
                
                hindcast_anchors_10.append(current_depth_at_start + change_10)
                hindcast_anchors_50.append(current_depth_at_start + change_50)
                hindcast_anchors_90.append(current_depth_at_start + change_90)
                hindcast_times.append(start_t + pd.Timedelta(hours=h))
            except Exception as e:
                pass
                
        if not hindcast_anchors_50:
            print("  Could not generate forecast (missing data)")
            continue
            
        # 2. Get Actual Data (History + Future)
        history_start = start_t - pd.Timedelta(days=2)
        end_t = start_t + pd.Timedelta(hours=72)
        
        if history_start < df_streamway.index.min(): history_start = df_streamway.index.min()
        if end_t > df_streamway.index.max(): end_t = df_streamway.index.max()
        
        actual_data = df_streamway.loc[history_start:end_t]
        
        # 3. Calculate Physics Trajectories for this event
        # Upper limit
        physics_upper_capacity = current_depth_at_start + MAX_TOTAL_RISE
        
        # Lower limit (drainage trajectory)
        physics_lower_drainage = []
        current_sim = current_depth_at_start
        forecast_times_for_physics = pd.date_range(start_t, end_t, freq='10min')
        for t in forecast_times_for_physics:
            physics_lower_drainage.append(current_sim)
            current_sim = max(MIN_DEPTH_ABSOLUTE, current_sim - MAX_DROP_PER_STEP)
        
        # 4. Create TWO-PANEL PLOT (like future forecast)
        fig = make_subplots(
            rows=2, cols=1,
            shared_xaxes=True,
            vertical_spacing=0.1,
            subplot_titles=(f'Hindcast: {p}th Percentile Event (Start: {start_t})', 'Historical Precipitation'),
            row_heights=[0.6, 0.4],
            specs=[[{"secondary_y": False}], [{"secondary_y": True}]]
        )
        
        # --- TOP PLOT: STREAM DEPTH ---
        
        # Historical Actual
        fig.add_trace(go.Scatter(
            x=actual_data.index,
            y=actual_data['streamway_depth_mm'],
            name='Actual',
            line=dict(color='blue', width=2)
        ), row=1, col=1)
        
        # Physics Limits
        # Check if we should show historical max
        forecast_max = max(hindcast_anchors_90) if hindcast_anchors_90 else 0
        if forecast_max > (MAX_DEPTH_OBSERVED * 0.8):
            fig.add_hline(
                y=MAX_DEPTH_OBSERVED,
                line_dash='dot',
                line_color='purple',
                opacity=0.3,
                annotation_text='Historical Max',
                row=1, col=1
            )
        
        # Upper capacity
        fig.add_hline(
            y=physics_upper_capacity,
            line_dash='dot',
            line_color='darkred',
            opacity=0.5,
            annotation_text='Physical Max Rise',
            row=1, col=1
        )
        
        # Lower drainage
        fig.add_trace(go.Scatter(
            x=forecast_times_for_physics,
            y=physics_lower_drainage,
            name='Physical Max Drainage',
            line=dict(color='darkblue', width=1, dash='dot'),
            opacity=0.5
        ), row=1, col=1)
        
        # Absolute floor
        fig.add_hline(
            y=MIN_DEPTH_ABSOLUTE,
            line_dash='dot',
            line_color='brown',
            opacity=0.3,
            annotation_text='Absolute Floor',
            row=1, col=1
        )
        
        # Forecast with Splines
        if len(hindcast_anchors_50) >= 4:
            from scipy.interpolate import CubicSpline
            hours_from_start = [(t - start_t).total_seconds() / 3600 for t in hindcast_times]
            
            spline_10 = CubicSpline(hours_from_start, hindcast_anchors_10)
            spline_50 = CubicSpline(hours_from_start, hindcast_anchors_50)
            spline_90 = CubicSpline(hours_from_start, hindcast_anchors_90)
            
            xs = np.linspace(0, max(hours_from_start), 200)
            ts = [start_t + pd.Timedelta(hours=x) for x in xs]
            
            ys_10 = spline_10(xs)
            ys_50 = spline_50(xs)
            ys_90 = spline_90(xs)
            
            # Confidence Interval
            fig.add_trace(go.Scatter(
                x=ts, y=ys_90,
                mode='lines',
                line=dict(width=0),
                showlegend=False
            ), row=1, col=1)
            
            fig.add_trace(go.Scatter(
                x=ts, y=ys_10,
                mode='lines',
                line=dict(width=0),
                fill='tonexty',
                fillcolor='rgba(255, 0, 0, 0.2)',
                name='10th-90th Percentile'
            ), row=1, col=1)
            
            # Median Forecast
            fig.add_trace(go.Scatter(
                x=ts, y=ys_50,
                name='Forecast (Median)',
                line=dict(color='red', dash='dash', width=2)
            ), row=1, col=1)
        
        # Anchors
        fig.add_trace(go.Scatter(
            x=hindcast_times,
            y=hindcast_anchors_50,
            name='Anchor Points',
            mode='markers',
            marker=dict(color='orange', symbol='diamond', size=8)
        ), row=1, col=1)
        
        # Danger Thresholds
        fig.add_hline(y=700, line_dash='dash', line_color='orange', annotation_text='Caution (700mm)', row=1, col=1)
        fig.add_hline(y=850, line_dash='dash', line_color='red', annotation_text='Danger (850mm)', row=1, col=1)
        
        # Start Line
        fig.add_vline(
            x=start_t.timestamp() * 1000,
            line_dash='dot',
            line_color='green',
            annotation_text='Start'
        )
        
        # --- BOTTOM PLOT: PRECIPITATION ---
        weather_data = df_merged.loc[history_start:end_t]
        
        # Precip Amount
        fig.add_trace(go.Bar(
            x=weather_data.index,
            y=weather_data['precip_15min'],
            name='Precipitation (mm)',
            marker_color='lightblue',
            opacity=0.7
        ), row=2, col=1, secondary_y=False)
        
        # Precip Probability (if available)
        if 'precip_prob' in weather_data.columns:
            fig.add_trace(go.Scatter(
                x=weather_data.index,
                y=weather_data['precip_prob'],
                name='Precip Probability (%)',
                line=dict(color='purple', width=1.5, dash='dot'),
                mode='lines'
            ), row=2, col=1, secondary_y=True)
        
        # Start line on precip plot too
        fig.add_vline(
            x=start_t.timestamp() * 1000,
            line_dash='dot',
            line_color='green'
        )
        
        # Layout
        fig.update_layout(
            template='plotly_white',
            height=900,
            hovermode='x unified',
            showlegend=True,
            legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
        )
        
        fig.update_yaxes(title_text='Depth (mm)', row=1, col=1)
        fig.update_yaxes(title_text='Precip (mm)', row=2, col=1, secondary_y=False)
        fig.update_yaxes(title_text='Probability (%)', row=2, col=1, secondary_y=True, range=[0, 105])
        
        fig.show()
else:
    print("Model 24h not found.")


Generating hindcasts for key percentiles (5th, 50th, 95th) of 24h changes...

Generating Hindcast for 5th Percentile (24h change: -344.00mm) at 2025-08-04 13:50:00



Generating Hindcast for 10th Percentile (24h change: -211.00mm) at 2025-08-28 16:20:00



Generating Hindcast for 50th Percentile (24h change: -10.00mm) at 2025-08-06 12:40:00



Generating Hindcast for 90th Percentile (24h change: 265.00mm) at 2025-08-03 21:20:00



Generating Hindcast for 95th Percentile (24h change: 438.00mm) at 2025-08-03 18:10:00


## 5. Recent & Future Forecast

We apply the model to recent data to see how it performs now.

In [11]:
# --- GENERATE FUTURE FORECAST USING ANCHOR & SPLINE (WITH QUANTILES & PHYSICS) ---

from scipy.interpolate import CubicSpline
import numpy as np

print("Generating 72-hour forecast (10th, 50th, 90th percentiles)...")

# Calculate Physics Constraints from Historical Data
print("Calculating physics constraints from historical data...")

# 1. Maximum Drainage Rate (per 10-minute step)
single_step_changes = df_streamway['streamway_depth_mm'].diff()
MAX_DROP_PER_STEP = abs(single_step_changes.min())

# 2. Maximum Total Rise (from depth change analysis)
max_rises = [hourly_rise_stats.get(f'{h}h', {}).get('max', 0) for h in range(1, 73)]
MAX_TOTAL_RISE = max([r for r in max_rises if not np.isnan(r)]) if max_rises else 2000.0

# 3. Absolute Depth Floor (historical minimum)
MIN_DEPTH_ABSOLUTE = df_streamway['streamway_depth_mm'].min()

# 4. Historical Maximum (for reference visualization only)
MAX_DEPTH_OBSERVED = df_streamway['streamway_depth_mm'].max()

print(f"  Max drainage rate: {MAX_DROP_PER_STEP:.1f} mm/10min")
print(f"  Max total rise: {MAX_TOTAL_RISE:.1f} mm")
print(f"  Historical min depth: {MIN_DEPTH_ABSOLUTE:.1f} mm")
print(f"  Historical max depth: {MAX_DEPTH_OBSERVED:.1f} mm")

def apply_physics_constraints(forecast_array, current_depth):
    """
    Apply physical constraints to prevent impossible predictions.
    
    forecast_array: Raw depth predictions (numpy array)
    current_depth: Starting water level (mm)
    
    Returns: Clamped forecast array
    """
    clamped_forecast = []
    last_depth = current_depth
    
    for pred in forecast_array:
        # A. Apply Capacity Ceiling (Max possible rise from current)
        if pred > (current_depth + MAX_TOTAL_RISE):
            pred = current_depth + MAX_TOTAL_RISE
            
        # B. Apply Drainage Speed Limit
        physically_lowest_possible = last_depth - MAX_DROP_PER_STEP
        if pred < physically_lowest_possible:
            pred = physically_lowest_possible
            
        # C. Apply Absolute Depth Floor (cannot go below historical minimum)
        if pred < MIN_DEPTH_ABSOLUTE:
            pred = MIN_DEPTH_ABSOLUTE
            
        clamped_forecast.append(pred)
        last_depth = pred
        
    return np.array(clamped_forecast)

# Get starting point
start_time = df_streamway['streamway_depth_mm'].last_valid_index()
start_depth = df_streamway.loc[start_time, 'streamway_depth_mm']

print(f"Starting from: {start_time}")
print(f"Current depth: {start_depth:.2f}mm")

# Generate predictions at all anchor hours
anchor_preds_10 = []
anchor_preds_50 = []
anchor_preds_90 = []
anchor_times = []

for h in sorted(anchor_models.keys()):
    model_info = anchor_models[h]
    models = model_info['models']
    features = model_info['features']
    
    if start_time not in df_merged.index:
        continue
        
    try:
        X_input = df_merged.loc[start_time:start_time, features]
        
        if X_input.empty or X_input.isna().any().any():
            continue
        
        # Predict depth change for each quantile
        change_10 = models['p10'].predict(X_input)[0]
        change_50 = models['p50'].predict(X_input)[0]
        change_90 = models['p90'].predict(X_input)[0]
        
        # Store prediction
        pred_time = start_time + pd.Timedelta(hours=h)
        anchor_times.append(pred_time)
        
        anchor_preds_10.append(start_depth + change_10)
        anchor_preds_50.append(start_depth + change_50)
        anchor_preds_90.append(start_depth + change_90)
        
    except Exception as e:
        continue

print(f"Generated {len(anchor_times)} anchor point predictions")

# Create spline interpolation for smooth 10-minute resolution
if len(anchor_times) >= 4:
    # Convert times to hours from start
    hours_from_start = [(t - start_time).total_seconds() / 3600 for t in anchor_times]
    
    # Create splines for each quantile
    spline_10 = CubicSpline(hours_from_start, anchor_preds_10)
    spline_50 = CubicSpline(hours_from_start, anchor_preds_50)
    spline_90 = CubicSpline(hours_from_start, anchor_preds_90)
    
    # Generate 10-minute resolution forecast
    max_hours = max(hours_from_start)
    forecast_hours = np.arange(0, max_hours + 0.167, 0.167)
    
    forecast_depths_10_raw = spline_10(forecast_hours)
    forecast_depths_50_raw = spline_50(forecast_hours)
    forecast_depths_90_raw = spline_90(forecast_hours)
    
    # Apply Physics Constraints
    print("Applying physics constraints...")
    forecast_depths_10 = apply_physics_constraints(forecast_depths_10_raw, start_depth)
    forecast_depths_50 = apply_physics_constraints(forecast_depths_50_raw, start_depth)
    forecast_depths_90 = apply_physics_constraints(forecast_depths_90_raw, start_depth)
    
    forecast_times = [start_time + pd.Timedelta(hours=h) for h in forecast_hours]
    
    # Create forecast DataFrame
    df_forecast = pd.DataFrame({
        'time': forecast_times,
        'predicted_depth': forecast_depths_50,
        'depth_10': forecast_depths_10,
        'depth_90': forecast_depths_90
    }).set_index('time')
    
    # Store anchor predictions for plotting
    anchor_predictions = anchor_preds_50
    
    # Calculate Physics Limit Trajectories for Visualization
    # Upper Limit (Capacity)
    physics_upper_capacity = start_depth + MAX_TOTAL_RISE
    
    # Lower Limit (Max Drainage)
    physics_lower_drainage = []
    current = start_depth
    for i in range(len(forecast_times)):
        physics_lower_drainage.append(current)
        current = max(MIN_DEPTH_ABSOLUTE, current - MAX_DROP_PER_STEP)
    
    # Store in df_forecast
    df_forecast['physics_upper_capacity'] = physics_upper_capacity
    df_forecast['physics_lower_drainage'] = physics_lower_drainage
    df_forecast['physics_floor_absolute'] = MIN_DEPTH_ABSOLUTE
    df_forecast['historical_max_ref'] = MAX_DEPTH_OBSERVED
    
    print(f"Generated smooth 10-minute forecast with confidence intervals and physics constraints")
    
else:
    print("Not enough anchor points for spline interpolation")
    df_forecast = pd.DataFrame()


Generating 72-hour forecast (10th, 50th, 90th percentiles)...
Calculating physics constraints from historical data...
  Max drainage rate: 192.0 mm/10min
  Max total rise: 2032.0 mm
  Historical min depth: 458.0 mm
  Historical max depth: 2652.0 mm
Starting from: 2025-11-22 00:40:00
Current depth: 630.00mm
Generated 72 anchor point predictions
Applying physics constraints...
Generated smooth 10-minute forecast with confidence intervals and physics constraints


In [12]:
# --- VISUALIZE FORECAST WITH PRECIPITATION & PROBABILITY & PHYSICS ---
from plotly.subplots import make_subplots

if not df_forecast.empty:
    # Create subplots
    fig = make_subplots(
        rows=2, cols=1, 
        shared_xaxes=True, 
        vertical_spacing=0.1,
        subplot_titles=("Streamway Depth Forecast (With Physics Constraints)", "Precipitation & Probability Forecast"),
        row_heights=[0.6, 0.4],
        specs=[[{"secondary_y": False}], [{"secondary_y": True}]]
    )
    
    # --- TOP PLOT: STREAM DEPTH ---
    
    # Historical data
    history_start = start_time - pd.Timedelta(days=2)
    hist_data = df_streamway.loc[history_start:start_time]
    
    fig.add_trace(go.Scatter(
        x=hist_data.index,
        y=hist_data['streamway_depth_mm'],
        name='Historical Depth',
        line=dict(color='blue', width=2)
    ), row=1, col=1)
    
    # Physics Limits (BEFORE confidence intervals, so they're in the background)
    
    # Historical Maximum (Reference only - show if forecast approaches it)
    historical_max = df_forecast['historical_max_ref'].iloc[0]
    forecast_max = df_forecast['depth_90'].max()
    
    # Show line only if forecast is within 80% of historical max
    if forecast_max > (historical_max * 0.8):
        fig.add_hline(
            y=historical_max,
            line_dash='dot',
            line_color='purple',
            opacity=0.3,
            annotation_text='Historical Max',
            row=1, col=1
        )
    
    # Upper Limit (Capacity from current depth)
    fig.add_trace(go.Scatter(
        x=df_forecast.index,
        y=df_forecast['physics_upper_capacity'],
        name='Physical Max Rise',
        line=dict(color='darkred', width=1, dash='dot'),
        mode='lines',
        opacity=0.5
    ), row=1, col=1)
    
    # Lower Limit (Max Drainage Rate)
    fig.add_trace(go.Scatter(
        x=df_forecast.index,
        y=df_forecast['physics_lower_drainage'],
        name='Physical Max Drainage',
        line=dict(color='darkblue', width=1, dash='dot'),
        mode='lines',
        opacity=0.5
    ), row=1, col=1)
    
    # Absolute Floor
    fig.add_hline(
        y=df_forecast['physics_floor_absolute'].iloc[0],
        line_dash='dot',
        line_color='brown',
        opacity=0.3,
        annotation_text='Absolute Floor',
        row=1, col=1
    )
    
    # Confidence Interval (10th - 90th)
    fig.add_trace(go.Scatter(
        x=df_forecast.index,
        y=df_forecast['depth_90'],
        mode='lines',
        line=dict(width=0),
        showlegend=False,
        name='90th Percentile'
    ), row=1, col=1)
    
    fig.add_trace(go.Scatter(
        x=df_forecast.index,
        y=df_forecast['depth_10'],
        mode='lines',
        line=dict(width=0),
        fill='tonexty',
        fillcolor='rgba(255, 0, 0, 0.2)',
        name='10th-90th Percentile',
        showlegend=True
    ), row=1, col=1)
    
    # Forecast (Median)
    fig.add_trace(go.Scatter(
        x=df_forecast.index,
        y=df_forecast['predicted_depth'],
        name='Forecast (Median)',
        line=dict(color='red', width=2, dash='dash')
    ), row=1, col=1)
    
    # Anchor points (Median)
    if len(anchor_times) > 0:
        fig.add_trace(go.Scatter(
            x=anchor_times,
            y=anchor_preds_50,
            name='Anchor Points',
            mode='markers',
            marker=dict(color='orange', size=8, symbol='diamond')
        ), row=1, col=1)
    
    # Danger thresholds
    fig.add_hline(y=700, line_dash="dash", line_color="orange", annotation_text="Caution (700mm)", row=1, col=1)
    fig.add_hline(y=850, line_dash="dash", line_color="red", annotation_text="Danger (850mm)", row=1, col=1)
    
    # --- BOTTOM PLOT: PRECIPITATION & PROBABILITY ---
    
    forecast_end = df_forecast.index.max()
    weather_data = df_merged.loc[history_start:forecast_end]
    
    # Precip Amount
    fig.add_trace(go.Bar(
        x=weather_data.index,
        y=weather_data['precip_15min'],
        name='Precipitation (mm)',
        marker_color='lightblue',
        opacity=0.7
    ), row=2, col=1, secondary_y=False)
    
    # Precip Probability
    if 'precip_prob' in weather_data.columns:
        fig.add_trace(go.Scatter(
            x=weather_data.index,
            y=weather_data['precip_prob'],
            name='Precip Probability (%)',
            line=dict(color='purple', width=1.5, dash='dot'),
            mode='lines'
        ), row=2, col=1, secondary_y=True)
    
    # Vertical line at "now"
    fig.add_vline(x=start_time.timestamp() * 1000, line_dash="dot", line_color="green", annotation_text="Now")
    
    # Layout
    fig.update_layout(
        title=f'Streamway Depth Forecast (Quantile Regression + Physics Constraints)<br><sub>Historical Max: {historical_max:.0f}mm</sub>',
        template='plotly_white',
        height=900,
        hovermode='x unified',
        showlegend=True,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    
    fig.update_yaxes(title_text="Depth (mm)", row=1, col=1)
    fig.update_yaxes(title_text="Precip (mm)", row=2, col=1, secondary_y=False)
    fig.update_yaxes(title_text="Probability (%)", row=2, col=1, secondary_y=True, range=[0, 105])
    
    fig.show()
else:
    print("No forecast data to visualize")
